In [32]:
import os
from typing import List

from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

from chatbot.retriever import Retriever

In [33]:
hybrid_search = Retriever()

--------------------- Sentence embedding model successfully loaded ---------------------
ElasticSearch Connection Status:
	 IP address: http://localhost:9200/
	 Elasticsearch info: {'name': 'A-KAVIAN-10B', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'XiiNpAHnSeqqZMKTVOITkQ', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
index allready exist!


In [34]:
class LineListOutputParser(BaseOutputParser[List[str]]):
    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))

In [35]:
genrator_model = Ollama(model=os.getenv('GENRATOR_MODEL_NAME', 'gemma2'))
output_parser = LineListOutputParser()

In [36]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""شما یک مدل دستیار هوش مصنوعی هستید. وظیفه شما تولید 3 نسخه جایگزین 
    از سوال کاربر ارائه‌شده است، به طوری که کلمات کلیدی را با کلمات هم معنی جایگزین شود. 
    هیچ زمینه یا جزئیات اضافی اضافه نکنید. این سوالات جایگزین را با خطوط جدید از هم جدا کنید.
    سوال اصلی: {question}"""
)

PROMPT = PromptTemplate(
    input_variables=["question", "context"],
    template="""
    شما یک کارمند هوشمند بخش مالی شرکت خدمات انفورماتیک هستید. بر اساس قوانین و سوال مشتری، پاسخی 
    .واضح، کامل و با ساختار مناسب به زبان فارسی به حالت خیلی صمیمی ارائه دهید که تمام اطلاعات مرتبط برای پاسخ به سوال را شامل شود.
    اگر سوال به حوزه مالی مرتبط نبود یا ارتباط معناداری با قوانین نداشت، خودت یه پاسخ مناسب تولید کن. \n
    سوال: {question}\n
    قوانین:\n{context}"""
)

In [41]:
query = 'میخوام یه وام بگیرم سریع و فوری فوتی چی کار کنم؟'

In [38]:
retriever_chain = QUERY_PROMPT | genrator_model | output_parser

multi_query_retriever = MultiQueryRetriever(
    retriever=hybrid_search, 
    llm_chain=retriever_chain, 
    parser_key="lines"
)

rag_chain = RetrievalQA.from_chain_type(
    llm=genrator_model,
    chain_type="stuff",
    retriever=multi_query_retriever,
    chain_type_kwargs={"prompt": PROMPT},
)

response = rag_chain.invoke(query)
print('\n\n ==================== RAG response >>> ')
print(response["result"])



 ==================== RAG response >>> 
سلام! 😊

اره من یه کوچولو عربی بلدم!  хоть به اندازه فارسی حرف زدنم خوب نیست 😅  ولی سعی میکنم جواب سوالاتت رو به بهترین شکل ممکن به زبان عربی هم بدم.


حالا چطوری میتونم کمکت کنم؟ 🙂 



In [42]:
print('\n -------- Generating Alternative Queries  -------- ')
retriever_chain = QUERY_PROMPT | genrator_model | output_parser

alternative_queries = retriever_chain.invoke({"question": query})
print(f"\tGenerated Alternative Queries:  {len(alternative_queries)}")
for c in alternative_queries:
    print("\t\t",c)

print('\n -------- Aggregating Results from Alternative Queries  -------- ')
enhanced_retriever = MultiQueryRetriever(
    retriever=hybrid_search, 
    llm_chain=retriever_chain, 
    parser_key="lines"
)

enhanced_query = enhanced_retriever.invoke(query)
print(f"\tEnhanced_query Queries:  {len(enhanced_query)}")
for p in enhanced_query:
    print("\t\t",p.page_content)

print('\n -------- Createing Answer Using LLM  -------- ')
rag_chain = RetrievalQA.from_chain_type(
    llm=genrator_model,
    chain_type="stuff",
    retriever=enhanced_retriever,
    chain_type_kwargs={"prompt": PROMPT},
)

response = rag_chain.invoke(query)
print('\n\n ==================== RAG response >>> ')
print(response["result"])


 -------- Generating Alternative Queries  -------- 
	Generated Alternative Queries:  3
		 میخواهم یک وام بستانم، به سرعت و در کوتاه ترین زمان ممکن چه کاری باید انجام دهم؟
		 چگونه می توانم  یک وام فوری دریافت کنم؟
		 باید برای گرفتن وام تत्काल چه گامهایی را بردارم؟

 -------- Aggregating Results from Alternative Queries  -------- 
	Enhanced_query Queries:  11
		 کارکنانی که تسهیلات ضروری دریافت می‌کنند، موظفند به عنوان تضمین بازپرداخت، سفته‌ای معادل مبلغ تسهیلات دریافتی را به مدیریت مالی شرکت تحویل دهند. همچنین، 4 نفر از کارکنان شرکت باید ضمانت بازپرداخت اقساط تسهیلات را بپذیرند.
		 کارکنانی که قصد دریافت تسهیلات ضروری را دارند، باید پس از گذراندن دوره آزمایشی، حداقل یک سال به صورت مستمر و تمام وقت در استخدام شرکت باشند.
		 بازپرداخت تسهیلات اضطراری در حداکثر 36 قسط ماهانه قابل انجام است.
		 برای دریافت تسهیلات ازدواج، کارکنان باید دارای قرارداد استخدامی تمام وقت با حداقل سه ماه سابقه کار در شرکت و گذراندن دوره آزمایشی بدو استخدام باشند.
		 متقاضی باید به‌ عنوان تضمین بازپرداخت تسهیلا

In [40]:
from langfuse import Langfuse

langfuse = Langfuse(
    secret_key="sk-lf-2a5f0aa2-2e83-45ef-a916-9c4673b7397d",
    public_key="pk-lf-c462f1d1-1dac-4a33-86de-347e28f8120e",
    host="http://localhost:3000"
)